# [75.06 / 95.58] Organización de Datos
## Trabajo Práctico 1: Análisis Exploratorio de Datos

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**https://github.com/fdelmazo/7506-datos**

Presentamos aca un análisis de datos obtenido de usuarios que visitaron www.trocafone.com, un sitio de e-commerce de compra y venta de celulares reacondicionados, con operaciones principalmente en Brasil.

## Set up inicial e información general del dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify # pip install squarify
import geopandas as gpd #  conda install -c conda-forge geopandas
from wordcloud import WordCloud # conda install -c conda-forge wordcloud
from shapely.geometry import Point
from time import strptime
import calendar
%matplotlib inline

df = pd.read_csv('data/events.csv', low_memory=False)

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['figure.figsize'] = (25,15)

In [ ]:
with pd.option_context('display.max_column',0):
  display(df.sample(n=5))

## Set up inicial e información general del dataset

In [ ]:
bytes_used = df.memory_usage().sum()
print('Memoria usada: {:.2f}MB'.format(bytes_used/1000000))
print('{} atributos y {} registros en el dataframe.\n'.format(df.shape[1],df.shape[0]))
print('Primer registro: {} \nÚltimo registro: {}.'.format(df['timestamp'].min(),df['timestamp'].max()))

In [ ]:
describe = df.describe().T
descripcion = pd.read_csv('data/columns-desc.csv',index_col='column')
data = pd.merge(descripcion,describe,left_index=True,right_index=True)
data['null count'] = df.isnull().sum()
data['dtype'] = df.dtypes
with pd.option_context('display.max_colwidth',-1):
    display(data)

**Se sabe que los datos proporcionados son un mero subconjunto de todos los datos de la empresa.**

**¿Cómo se decidio el truncamiento de la base original?**

In [ ]:
print('Cantidad de usuarios: {}'.format(df['person'].nunique()))

by_person = df[['event','person']].groupby('person')
con_checkouts = by_person.agg({'event':lambda x: any(y == 'checkout' for y in x)}).sum()['event']

print('Cantidad de usuarios con checkouts: {}'.format(con_checkouts))

Después de varias teorías y sus respectivas pruebas, encontramos que todos los usuarios presentes en el set tienen al menos un evento checkout, mostrando que efectivamente la base de datos original se truncó (ya que sería inocente creerse que el 100% de los usuarios que entran al sitio tienen un checkout). 

Este dato parece menor pero es muy importante; nos muestra que las conclusiones de este análisis no podrán hacerse sobre el sitio en su totalidad. Por los limites de los datos de entrada todas las conclusiones serán sobre los usuarios que hayan tenido al menos un checkout, en vez de acerca de todos los usuarios del sitio.

## Curado y limpieza de datos

Pasado el vistazo general a los datos, se pueden transformar datos en sus tipos correspondientes para mejor manejo de estos y para ahorro de memoria.

In [ ]:
# Los atributos con pocos valores posibles se pasan a variables categoricas para ahorrar memoria
df['event'] = df['event'].astype('category')
df['condition'] = df['condition'].astype('category')
df['storage'] = df['storage'].astype('category')
df['search_engine'] = df['search_engine'].astype('category')
df['channel'] = df['channel'].astype('category')
df['device_type'] = df['device_type'].astype('category')

# El tiempo es mejor manejarlo como tal
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Chequeo
df.info()

In [ ]:
ahorro = (bytes_used - df.memory_usage().sum())
porcentaje = (ahorro/bytes_used) * 100
print("Memoria ahorrada: {:.4f}MB ({:.2f}%)".format(ahorro/1000000,porcentaje))

Se decide usar el tiempo del evento registrado como un id de este (es decir, como índice del set de datos).

In [ ]:
df.set_index('timestamp', drop=False, inplace=True)

TO DO:
    - Sacar registros producto del 'double tracking'. Por ej: Compras iguales (usuario, modelo) registradas en el mismo minuto
    

## Data Mining

De las columnas `model`, `operating_system_version` y `browser_version` se pueden extraer datos de la marca, el sistema operativo y el explorador. Estos se usan para generar nuevos dataframes y luego se hace un *left join* de los datos obtenidos.

Todo el código respecto a la creación de nuevos dataframes está ubicado en `anexo.ipynb`.

In [ ]:
df_brands = pd.read_csv('data/brands.csv')
df = df.merge(df_brands, how='left', on='model')
df['brand'].astype('category')
sample = df[(df['model'].notnull()) & (df['brand'].notnull())]
sample[['model','brand']].head()

In [ ]:
df_os = pd.read_csv('data/os.csv')
df = df.merge(df_os, how='left', on='operating_system_version')
df['operating_system'].astype('category')
sample = df[(df['operating_system_version'].notnull()) & (df['operating_system'].notnull())]
sample[['operating_system_version', 'operating_system']].head()

In [ ]:
df_browsers = pd.read_csv('data/browsers.csv')
df = df.merge(df_browsers, how='left', on='browser_version')
df['browser'].astype('category')
sample = df[(df['browser_version'].notnull()) & (df['browser'].notnull())]
sample[['browser_version','browser']].head()

Extracción de información de las fechas.

In [ ]:
df['month_number'] = df.index.month
df['month_name'] = df['month_number'].apply(lambda x: calendar.month_abbr[x])
df['week_day'] = df.index.weekday
df['week_day_name'] = df.index.weekday_name
df['day_date'] = df.index.to_period('D')
df['day_dom'] = df.index.day
df['day_doy'] = df.index.dayofyear

---
## Análisis de eventos

* ¿Qué evento es el mas frecuente?
* ¿Están los eventos distribuidos uniformemente segun el día del mes?
* ¿Están los eventos distribuidos uniformemente segun el día de la semana?
* ¿En qué mes hubo mas visitas al sitio?

In [ ]:
event = df['event']
descripcion = pd.read_csv('data/events-desc.csv',index_col='event')
descripcion['value_counts'] = event.value_counts()
with pd.option_context('display.max_colwidth',0):
    display(descripcion)

In [ ]:
orden = event.value_counts().head(7).index
visu = sns.countplot(x='event',data=df,order=orden)
visu.axes.set_title('Frecuencia de eventos')
visu.set_xlabel("Evento")
visu.set_ylabel("Cantidad")

plt.savefig('informe/figures/01-eventos-barplot.png')

El evento con mas hits es el de ver un producto. Esto tiene sentido ya que trocafone es una plataforma de e-commerce y ver productos es su principal función como sitio. 

In [ ]:
data = df.pivot_table(index='day_dom',columns='month_number', values='event', aggfunc='count')
data = ((data-data.min()) / (data.max() - data.min()))

visu = sns.heatmap(data.T,  cmap="OrRd")
visu.set_title("Tráfico en sitio según mes y día")
visu.set_xlabel("Día")
visu.set_ylabel("Mes")

plt.savefig('informe/figures/02-eventos_segun_mes-heatmap.png')

Como se puede ver, no hay relación directa entre dia del mes y mayores visitas al sitio. Se podría decir que la segunda quincena tiene más visitas que la primera, pero las magnitudes no son lo suficientemente distintas como para argumentar esto.

Es importante que este gráfico este normalizado para poder encontrar patrones y no simplemente ver como el mes con más eventos es simplemente el mes con más eventos por día.

In [ ]:
data = df.pivot_table(index='week_day',columns='month_number', values='event', aggfunc='count')
data.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data = ((data-data.min()) / (data.max() - data.min()))

visu = sns.heatmap(data.T,  cmap="OrRd")
visu.set_title("Tráfico (normalizado) en sitio según mes y día de la semana")
visu.set_xlabel("Día de la semana")
visu.set_ylabel("Mes")

plt.savefig('informe/figures/03-eventos_segun_dow-heatmap.png')

Se puede observar en el gráfico que los días de mayor tráfico son los días laborales mientras que el fin de semana los usuarios visitan notoriamente menos la página.

In [ ]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
month_counts = df.groupby('month_name').count()
month_counts = month_counts.loc[month_name]
visu = month_counts['event'].plot(kind='bar')
visu.axes.set_title('Cantidad de visitas segun mes')
visu.set_xlabel("Mes")
visu.set_ylabel("Cantidad")

plt.savefig('informe/figures/04-eventos_segun_mes-barplot.png')

Como se puede ver, mayo y junio tienen considerablemente más visitas que el resto de los meses. Y esto es incluso teniendo en cuenta que en junio se registraron la mitad de los días (el último registro del set de datos es el 15 de junio).

Esto nos lleva directamente a la próxima sección.

### ¿Por que mayo y junio son los meses de mayor tráfico? ¿Eso implica una mayor cantidad de ventas?

In [ ]:
df_top = df.loc[(df['month_name'] == 'May') | (df['month_name'] == 'Jun')]

df_temporal = df_top[['event', 'day_dom']]
df_temporal = df_temporal.loc[(df_temporal['event'] == 'conversion') | (df_temporal['event'] == 'checkout') | (df_temporal['event'] == 'viewed product')]
df_temporal = df_temporal.groupby('day_dom')['event'].value_counts().unstack('event')

visu = plt.plot(np.log(df_temporal))
plt.legend(iter(visu), ('checkout', 'conversion', 'viewed_products'))
#visu.set_title("Tráfico en sitio según mes y día")
#visu.set_xlabel("Día")
#visu.set_ylabel("Mes")
#plt.savefig('informe/figures/05-???-lineplot.png')

TO DO:
    - Pequeña conclusion
    - Titulo, ejes

---

## Análisis geográfico

* ¿Desde que países se accede más al sitio?
* De esos paises, ¿desde que ciudades?

In [ ]:
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
data = countries.head(3)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['green','red','cyan'])
visu.set_title('Países con más visitas')

plt.savefig('informe/figures/06-paises_visitas-treemap.png')

Al ser Trocafone una empresa que inicialmente radicó en Brasil y llegó a Argentina en 2016 era esperable que Brasil sea el país de mayor tráfico.

Por la amplia diferencia, se separa a Brasil del gráfico y se analiza el tráfico en el resto de los países, para pdoer ver su diferencia en orden y magnitud.

In [ ]:
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
countries = countries.drop('Brazil')
data = countries.head(7)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.set_title('Países con más visitas, exceptuando Brazil')

plt.savefig('informe/figures/07-paises_visitas_sin_brazil-treemap.png')

TO DO:
    - Analisis: Reflexionar por qué tanta gente de USA entra a Trocafone

**Sacando las longitudes y latitudes de distintas ciudades del mundo, podemos ver que ciudades de los países con más trafico son las que más visitas tienen.** 

Con ayuda del módulo geopandas, podemos plotear directamente sobre un mapa.

Las bases de datos adicionales fueron sacadas de http://www.geonames.org/.

In [ ]:
BR = pd.read_csv('data/BR.csv', low_memory=False, sep='\t')
BR = BR[['name','latitude','longitude']]

# Para que geopandas pueda leer bien las latitudes y longitudes, deben ser de la clase Point
BR['coordinates'] = list(zip(BR['longitude'],BR['latitude']))
BR['coordinates'] = BR['coordinates'].apply(Point)
BR.head()

In [ ]:
# Se debe hacer un join de los datos que se tienen (nombre de ciudad, cantidad de eventos) y los datos de geonames (nombre de ciudad, punto en el mapa), y esto plotearlo sobre los datos de geopandas (nombre de pais, punto en el mapa mundial)

# Se preparan los datos para el join (inner join de nombre de ciudad (columna name))
ciudades_brazil = df.loc[(df['country'] == 'Brazil')]
data = ciudades_brazil['city'].value_counts()
data = data.drop('Unknown')
data = data.to_frame()
data.reset_index(inplace=True)
data = data.rename(columns={'index':'name','city':'count'})

# Se pasa de un dataframe normal de pandas a uno de geopandas
BRA = gpd.GeoDataFrame(BR, geometry='coordinates')

# Se hace el inner join de ambos sets. Siendo que geonames daba mucha más información de la necesaria, duplicando valores por ciudades, se borran los duplicados
data = BRA.merge(data, on='name')
data = data.drop_duplicates('name')

# Se prepara el 'fondo' del gráfico, siendo este nomás el país. Para esto se usan los datos por defecto de geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.name=='Brazil'].plot(color='white',edgecolor='black')

visu = data.plot(ax=ax,cmap='OrRd', legend=True)
visu.axes.set_title('Ciudades de Brazil con más visitas')

plt.savefig('informe/figures/08-ciudades_brazil-choropleth.png')

TO DO:
    - Cloropeth de regiones
    - Analisis

---
## Análisis de busquedas

Para lo que respecta a busquedas, hay varias cosas para analizar y encontrar:

1. La mas obvia, qué terminos ingresan los usuarios en el buscador? (columna search term)
    * Esto nos da una visualización rápida de los gustos en crudo de los usuarios. Lo que antojan.
1. Qué productos buscan específicamente, usando la interfaz? (evento searched product)
    * Esta busqueda es un poco más refinada, permitiendonos saber específicamente los productos buscados, en vez de la crudeza de escribir cualquier termino que nos puede plantear el punto anterior.
    
Si bien tienen un objetivo en comun, difieren sustancialmente los analisis. En el primero se puede ver una vista rápida de marcas/modelos, mientras que en la segunda habrá una idea un poco mas profunda.

In [ ]:
# Buscamos las palabras más buscadas, con un mínimo de 300 busquedas

search_terms = df['search_term'].dropna()
search_terms = search_terms.apply(lambda x: x.lower())
search_terms = search_terms.value_counts()
search_terms = search_terms[search_terms >= 300]

# Para que funcione correctamente el módulo de wordcloud, hay que juntar todas las palabras en el mismo texto.
text = ''
for w,q in zip(search_terms.index,search_terms):
    text += ' '.join([w for x in range(q)])

text = ' '.join([s for s in text.split() if len(s)>2])    

wordcloud = WordCloud(width=2000, height=800, margin=0,collocations=False).generate(text)
 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

plt.savefig('informe/figures/09-search_terms-wordcloud.png')

Se puede ver que los términos más buscados estan relacionados a las marcas de Motorola, Apple y Samsung.

In [ ]:
# Sabemos que el evento searched products refiere a varios SKUs. Spearemos y busquemos especificamente cuales son los más buscados.
searched = df[df['event']=='searched products']
skus_codiciados = searched['skus'].str.split(',').to_frame()
skus_codiciados = skus_codiciados['skus'].apply(pd.Series).unstack().reset_index()
skus_codiciados = skus_codiciados[['timestamp',0]]
skus_codiciados.rename(columns={0: 'sku'}, inplace=True)
skus_codiciados.sample(5)

In [ ]:
top_5 = skus_codiciados['sku'].value_counts().head(5)
visu = top_5.plot(kind='bar', title ="5 productos más buscados por interfaz")
visu.set_xlabel("SKU")
visu.set_ylabel("Busquedas")
plt.show()

plt.savefig('informe/figures/10-skus_buscados-barplot.png')

TO DO:
    - Agregar entre parentesis el nombre del modelo ademas del sku

## Análisis de modelos

Queremos encontrar algun patrón de visita y compra desde el punto de vista del dispositivo y luego ver si en esos patrones hay diferencias según marca o modelo

In [ ]:
# Qué eventos tenemos asociados a un modelo?
cronologia = df.loc[df['model'].notnull(), ['model', 'event','condition']]
cronologia['event'].value_counts()

Tomamos como cronología de eventos sobre un modelo:
    1. Visitarlo (`viewed product`)
    1. Mandar a comprarlo (`checkout`)
    1. Efectivamente comprarlo (`conversion`)

In [ ]:
# Se descartan los eventos 'lead' porque no influyen en el análisis
cronologia = cronologia.loc[cronologia['event'] != 'lead'] 

# Comparemos los modelos por los cuales más se opera segun el evento
vistos = cronologia.loc[cronologia['event']=='viewed product']['model'].value_counts().head()
checkouts = cronologia.loc[cronologia['event']=='checkout']['model'].value_counts().head()
comprados = cronologia.loc[cronologia['event']=='conversion']['model'].value_counts().head()

print("Más vistas") ; display(vistos)
print("Más checkouts") ; display(checkouts)
print("Más compras") ; display(comprados)

In [ ]:
# Definimos un set de X modelos como los más prominentes, haciendo una combinación de los 7 más vistos, los 5 más por comprar  y los 5 más comprados
modelos_prominentes = set(vistos.index)
modelos_prominentes.update(checkouts.index)
modelos_prominentes.update(comprados.index)

eventos = cronologia.groupby('model')['event'].value_counts().unstack('event') 
eventos = eventos[['viewed product', 'checkout', 'conversion']]
eventos = eventos.loc[eventos.index.isin(modelos_prominentes)]
eventos

In [ ]:
# Paso a porcentajes las columnas
eventos['total'] = eventos.sum(axis=1)
for c in eventos:
    eventos[c+'%'] = ( eventos[c] / eventos['total'] ) * 100

vistos = eventos['viewed product%']
checkouts = eventos['checkout%']
comprados = eventos['conversion%']

plt.bar(vistos.index, vistos, color=sns.xkcd_rgb["muted blue"],log=True)
plt.bar(checkouts.index, checkouts, bottom=vistos, color=sns.xkcd_rgb["muted green"],log=True)
plt.bar(comprados.index, comprados, bottom=vistos+checkouts, color=sns.xkcd_rgb["muted pink"],log=True)
plt.legend(['viewed product','checkout', 'conversion'])
plt.ylabel('Cantidad de eventos')
plt.xlabel('Modelos de celular')
plt.title('Cantidad de eventos en función de modelo de celular (escala logarítmica)')
plt.show()
plt.savefig('informe/figures/11-modelos_eventos-stackedbarplot.png')

In [ ]:
for c in eventos[['viewed product','checkout','conversion']]:
    eventos[c+' ranking'] = eventos[c].rank('index',ascending=False)

rankings = (eventos.filter(regex='ranking')).T
orden = (eventos.sort_values('viewed product ranking', ascending=True)).index
orden = [None] +[(str(i+1) + ' - ' + x) for i,x in enumerate(orden)]

fig, ax = plt.subplots()
ax.plot(rankings)
fig.gca().invert_yaxis()
plt.ylabel('Modelo')
plt.xlabel('Evento')
plt.title('Rankings de modelos para distintos eventos')
ax.set_yticklabels(orden)

plt.savefig('informe/figures/12-rankings_celulares-rank.png')

Es interesante ver como los celulares Samsung tienden a tener el comportamiento inverso al de los celulares Apple. Los Samsung son menos buscados pero más comprados mientras que los Apple son muy buscados pero poco comprados (¿será por 'espantarse' al ver el precio?).

Buscamos ahora alguna diferencia en compras de celulares re-acondicionados segun la condición de estos.

In [ ]:
condiciones_analizadas = ['Bom', 'Muito Bom', 'Excelente']

condicion = cronologia.groupby('model')['condition'].value_counts().unstack('condition')
condicion = condicion[condiciones_analizadas]
condicion = condicion.loc[condicion.index.isin(modelos_prominentes)]

condicion['total'] = condicion.sum(axis=1)
for c in condicion:
    condicion[c] = ( condicion[c] / condicion['total'] ) * 100

bom = condicion['Bom']
muito_bom = condicion['Muito Bom']
excelente = condicion['Excelente']

plt.bar(bom.index, bom, color=sns.xkcd_rgb["muted blue"],log=True)
plt.bar(muito_bom.index, muito_bom, bottom=bom, color=sns.xkcd_rgb["muted green"],log=True)
plt.bar(excelente.index, excelente, bottom=bom+muito_bom, color=sns.xkcd_rgb["muted pink"],log=True)
plt.legend(condiciones_analizadas)
plt.ylabel('Cantidad')
plt.xlabel('Modelo')
plt.title('Cantidad de vistas y compras segun condición de modelo')
ax.set_yticklabels(orden)


plt.savefig('informe/figures/13-condicion-stackedbarplot.png')

Vemos como nuevamente hay una diferencia substancial entre celulares marca Apple y celulares marca Samsung. Para los Samsungs, los usuarios estan más dispuestos a ver o comprar celulares en peor condición, mientras que para los Apple lo que más se buscan son celulares en mejor condición.

## Análisis de páginas estáticas

In [ ]:
df['staticpage'].value_counts()

Podemos ver que la cantidad de visitas a **Customer Service** es mucho mayor a la página de **FAQ**. Para optimizar recursos sería más eficiente intentar de redireccionar el tráfico a FAQ, haciendo más visible los links a la página y de ser necesario mejorándola.

In [ ]:
faq_and_service = df[(df['staticpage'] == 'CustomerService') | (df['staticpage'] == 'FaqEcommerce')]['staticpage']
visu = sns.countplot(faq_and_service)
visu.set(xlabel='Page', ylabel='Visits')
visu.axes.set_title('Cantidad de eventos según staticpage')

## Análisis de New vs Returning

La intención de este  análisis es mostrar que cantidad de usuarios solo ingresan una vez al site.
El primer gráfico no nos aporta información, porque todos los usuarios que volvieron a ingresar, en su primera vez fueron **New**.

In [ ]:
order = df['new_vs_returning'].value_counts().index
visu = sns.countplot(df['new_vs_returning'].dropna(), order=order)
visu.set(xlabel='Tipo de usario', ylabel='Visitas al home')
visu.axes.set_title('Cantidad de eventos según marca')

En este gráfico sí se puede apreciar los valores reales de usuarios que regresan con los que sólo entraron una vez.
Para aumentar la tasa de personas que regresan a ĺa página proponemos aumentar el presupuesto en publicidad y mejorar la experiencia de usuario de la home para que provea al usuario una experiencia más amena.

In [ ]:
gb = df.groupby(['person', 'new_vs_returning']).agg({'new_vs_returning' : 'size'})
gb = gb.unstack(level='new_vs_returning')
gb = gb['new_vs_returning']
returners_count = gb['Returning'].count()
only_once_count = gb['New'].count() - returners_count

objects = ('Regresan', 'Sólo una vez')
y_pos = np.arange(len(objects))
type_users_count = pd.Series([returners_count, only_once_count])
 
plt.bar(y_pos, type_users_count, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Cantidad')
plt.title('Tipos de usuarios')
 
plt.show()

## Análisis de Brands

In [ ]:
order = df['brand'].value_counts().index
visu = sns.countplot(df['brand'].dropna(), order=order)
visu.axes.set_title('Cantidad de eventos segun marca')
visu.axes.set_xlabel('Cantidad', fontsize=28);
visu.axes.set_ylabel('Marca', fontsize=28);
sns.set(font_scale=2)

In [ ]:
df_conv_by_brand = df.loc[(df['event'] == 'conversion') | (df['event'] == 'checkout')][['event', 'brand']]
# Es necesario para que no me muestre las otras categorías que ya fueron filtradas
df_conv_by_brand['event'] = df_conv_by_brand['event'].astype('object').astype('category')
ax = sns.countplot(x='brand', hue='event', data=df_conv_by_brand)
ax.set_yscale('log')

ax.legend(loc='upper left', fontsize=20,bbox_to_anchor=(0, 1.1))
ax.set_xlabel('Cantidad (log)', fontsize=28);
ax.set_ylabel('Marca', fontsize=28);
ax.set_title('Relación de conversiones en escala logarítmica', fontsize=28);
sns.set(font_scale=3)

## Análisis de tipos de dispositivos

Cómo la mayoría del tráfico proviene de smartphones y computadoras, debería dedicarse una mayor cantidad de recursos a desarrollar la aplicación para estos dispositivos y no dedicar mucho tiempo y desarrolladores a las aplicaciones para tablets.

In [ ]:
order = df['device_type'].value_counts().index
visu = sns.countplot(df['device_type'].dropna(), order=order)
visu.set(xlabel='Device Type', ylabel='Visits')
visu.axes.set_title('Cantidad de eventos segun tipo de dispositivo')

## ¿Aumentó Trocafone el presupuesto en publicidad en algún período específico?

In [ ]:
publicitados = df.loc[df['campaign_source'].notnull(), ['campaign_source','month']] #(df['event']).isnull().sum() = 0

publicitados['month'] = publicitados['month'].apply(lambda x: calendar.month_abbr[x])
month_counts = publicitados.groupby('month').count()
month_counts = month_counts.loc[month_name]
month_counts = month_counts['campaign_source']
plt.bar(month_counts.index, month_counts, color=sns.xkcd_rgb["muted green"])
plt.title('Cantidad de visitas por mes por una publicidad')

No se halló la respuesta buscada ya que la distribución presentada de la cantidad de visitas de campaign_hit remite a la misma distribución de eventos por usuario. Mayo es el mes que notoriamente tiene una mayor cantidad de eventos generalmente pero también de campaign_hits. Por lo tanto concluimos que el gráfico es irrelevante para este tipo de análisis.

## ¿Cuáles son los métodos de publicidad más usados?

In [ ]:
ranking = df['campaign_source'].value_counts()
ranking_visu = ranking.head(5)

visu = squarify.plot(ranking_visu, label=ranking_visu.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('5 metodos de publicidad más clikeados')

Por su importancia mundial era esperable que Google sea el método de publicidad con más alcance. Se procede a analizar los otros motores de b

In [ ]:
ranking = ranking.drop('google')
ranking_visu2 = ranking.head(10)

ranking_visu2
visu = squarify.plot(ranking_visu2, label=ranking.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('10 metodos de publicidad más clikeados sin Google')

## ¿Cuál es la distribución de la cantidad de eventos producidos por usuario en función de los eventos?

In [ ]:
usuarios = df.groupby(['person', 'event'])
usuarios = usuarios.size().unstack(level = 'event')
usuarios = usuarios.fillna(usuarios.mean())
p = sns.boxplot(data=usuarios)
sns.set(font_scale=1.5)

Como se puede observar el gráfico no es ilustrativo de la respuesta que se desea obtener. Por lo tanto, se propone truncar la éscala del eje y a un valor en el cual se puedan observar las figuras de manera apropiada y representativa.

In [ ]:
usuarios = df.groupby(['person', 'event'])
usuarios = usuarios.size().unstack(level = 'event')
usuarios = usuarios.fillna(usuarios.mean())
p = sns.boxplot(data=usuarios)
p.axes.set_ylim((0,25))


Se propone analizar la cantidad de eventos producidos por usuario en un gráfico de tipo boxplot. Se puede observar que los usuarios suelen más comúnmente ver los productos antes que comprarlos, algo que podía predecirse anteriormente. Lo que puede resultar llamativo es que la cantidad de usuarios que ven productos es mayor a los que los buscan, pero esto se puede explicar por el hecho de que en una búsqueda pueden verse varios productos a la vez y eso cuenta como un solo evento. En cambio, ante el evento viewed products al ver un producto se contabiliza como un solo evento.

In [ ]:
df['color'].unique()

In [ ]:
df.columns

In [ ]:
df['operating_system_version'].unique()

---
---

# TO DO 

Buenos robos:
- https://jilt.com/blog/ecommerce-metrics/
- https://python-graph-gallery.com/
- https://github.com/kevinspasiuk/orgaDatos/blob/master/General.ipynb -> Reloj / Postulaciones y vistas a lo largo del tiempo
- https://github.com/GastonMontes/75.06-Datos-TP1/blob/master/Notebooks/Junto.ipynb -> Sabe analizar
- Apunte cap 2 tiene muy buenos analisis
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Movies%20from%20IMDB.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/MovieLens.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Airplane%20Crashes.ipynb
- https://github.com/lrargerich/7506/blob/master/Notebooks/Analisis%20Exploratorio/Universities%20-%20Times%20ranking.ipynb
- https://github.com/idontdomath/datos-exploratory-data-analysis/blob/master/2018-02/02-kickstarter-projects.ipynb

---
- [X] ¿Pudieron descubrir features en el campo ‘model’? ¿Cuales fueron?
- [ ] ¿Identificaron patrones o funnels de usuarios que realizan checkouts/conversiones en Trocafone? 
    - [ ] ¿Se comportan de forma distinta dependiendo del tipo de dispositivo desde el cual acceden?
    - [ ] ¿Se comportan de forma distinta dependiendo del tipo de fuente de tráfico al que pertenecen?
- [X] ¿Realizaron algún análisis sobre búsquedas que realizan los usuarios y las keywords que utilizan apoyándose en algún tipo de visualización?
- [X] ¿Realizaron algún análisis de lugar donde se originan las visitas de los usuarios de Trocafone (a nivel país, regiones más importantes o ciudades más importantes) apoyándose en algún tipo de visualización?
- [ ] ¿Pudieron descubrir features jerarquizando información de alguno de los campos (por ejemplo “screen_resolution”)?

---
- [X] Boxplot
- [ ] Violin
- [X] Stacked bar
- [ ] Histograma
- [ ] Scatter plot
- [X] Radar chart
- [X] Visualicemos los modelos más prominentes por ranking
- [X] Trafico y compras en mismo grafico de lineas que no se como se llama y es igual (lineal?)
- [ ] Treemap barchart por meses y compras en cada mes. Normalizar el mes de junio!!
- [ ] Grafico de rankings de celular mas viewed pasados los meses
- [ ] Conversiones vs Checkouts 
- [ ] Analisis de mayo. Por que hay tantos datos?
- [X] Comparacion entre conversiones, checkouts y viewed products. 
- [ ] Agregar barplot de regiones antes del mapa b?razuca
---
- [ ] ¿Tienen todos los ejes su rótulo?
- [ ] ¿Tiene cada visualización un título?
- [ ] ¿Es entendible la visualización sin tener que leer la explicación?
- [ ] ¿El uso del color es adecuado?
- [ ] ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
- [ ] ¿Presenta el grupo un listado de "insights" aprendidos sobre los datos en base al análisis realizado? ¿Es interesante?